In [9]:
# TODO
# catologue risks
# refer to hse (health and safety executive)
# We have caught carp, barbel, roach, rudd, perch, skimmer, bream, gudgeon, tench,
#

In [10]:
from IPython.display import display, Markdown

In [11]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [12]:
from typing import Literal
from pydantic import BaseModel, Field


class Risk(BaseModel):
    """Describes a risk."""

    hazard: str = Field(
        description=(
            "Sentence describing the hazard. "
            "A hazard is something that may cause harm or damage."
        )
    )
    risks: list[str] = Field(
        description=(
            "Risks associated with the hazard. "
            "A risk is the harm that may occur from the hazard. "
            "Describe each risk in a sentence."
        )
    )
    affects: Literal["Young people", "Leaders", "All present"] = Field(
        description="Who is at risk. Leaders are adult Scout leaders."
    )
    controls: list[str] = Field(
        description=(
            "Controls are ways of making the activity safer by removing or reducing the "
            "risk. For example, you may use a different piece of equipment or you might "
            "change the way you do the activity."
        )
    )

    def __str__(self) -> str:
        return "\n".join(
            [
                "### Hazard",
                f"{self.hazard}",
                "### Affects",
                self.affects,
                "### Risks",
                "\n\n".join([r for r in self.risks]),
                "### Controls",
                "\n\n".join([c for c in self.controls]),
                "\n\n",
            ]
        )


class Risks(BaseModel):
    """List of risks."""

    title: str = Field(description="Title for the activity.")
    risks: list[Risk]

    def __str__(self) -> str:
        return "\n\n".join(
            [
                f"## {self.title}",
                "\n\n".join([str(r) for r in self.risks]),
            ]
        )

In [13]:
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate


system = """
A scout leader planning an activity for Scouts (aged 10-14) needs to perform a 
risk assessment for an activity at a given location.

Your task is to perform a risk assessment identifying hazards and the risks
associated with them.

When identifying hazards and risks, pay attention to the location as this can 
introduce additional hazards.

Always consider the impact that poor behaviour of young people could have.

Use plain British english and commonly used terms in your response. 
"""

examples = []

user_prompt = """
Location: {location}

Activity: {activity}

Identify {n} risk(s). Rank them from highest impact to lowest impact.
"""

examples = [
    HumanMessage(
        user_prompt.format(
            location="The scout hut is large brick building with a hard wooden floor and a raised stage at the front. I contains a number of pillars to support the roof and stacked chairs around the edges of the hall.",
            activity="Variety of indoor games such as dodgeball, crab football etc.",
            n=1,
        )
    ),
    AIMessage(
        content="",
        name="call_01",
        tool_calls=[
            {
                "name": "risks",
                "args": {
                    "title": "Indoor games at the Scout Hut",
                    "risks": [
                        {
                            "hazard": "Use of hot surfaces such as the stove or frying pan.",
                            "risks": [
                                "Scouts can get burned if they touch hot surfaces or spill hot oil.",
                                "Severe burns may require medical attention.",
                            ],
                            "affects": "Young people",
                            "controls": ["Explain and demonstrate how to use the cooking equipment"]
                        },
                    ]
                },
                "id": "1",
            }
        ],
    ),
    # Most tool-calling models expect a ToolMessage(s) to follow an AIMessage with tool calls.
    ToolMessage("", tool_call_id="01"),
    # Some models also expect an AIMessage to follow any ToolMessages,
    # so you may need to add an AIMessage here.
]

prompt = ChatPromptTemplate.from_messages(
    [("system", system), ("placeholder", "{examples}"), ("human", user_prompt)]
)

In [17]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)
llm = model.with_structured_output(Risks, include_raw=True)
app = prompt | llm

risks = app.invoke(
    {
        # "activity": "Pumpkin carving with tee light candles. Pumpkin carving will be done on tables in groups of 2, 2 groups per table.",
        "activity": "Variety of indoor games such as dodgeball, crab football etc.",
        # "activity": "Visiting 2 local chip shops and sample the chips. Scouts will need to cross roads and not get lost. Scouts will travel in groups of 4, each with one adult leader. Scouts should plan.",
        # "activity": "Scouts making pancakes in the kitchen when being supervised by 2 adults.",
        # "location": "The scout kitchen is a medium sized kitchen similar to a typical family kitchen",
        "location": "The Queens Hall is large brick building with a hard wooden floor and a raised stage at the front. I contains a number of exposed pillars to support the roof and stacked chairs.",
        # "location": "Local urban area at night.",
        "n": 7,
    }
)

# TODO would generally run this generation several times and filter

In [18]:
print(risks["parsed"].model_dump_json(indent=3))

{
   "title": "Indoor Games at The Queens Hall",
   "risks": [
      {
         "hazard": "Players running into exposed pillars.",
         "risks": [
            "Injury from collision with pillars, such as bruises or fractures.",
            "Potential for head injuries if players are running fast."
         ],
         "affects": "Young people",
         "controls": [
            "Mark the pillars with bright tape to increase visibility.",
            "Set clear boundaries for the play area to avoid the pillars."
         ]
      },
      {
         "hazard": "Slipping on the hard wooden floor.",
         "risks": [
            "Injury from falls, such as sprains or fractures.",
            "Potential for head injuries if falling awkwardly."
         ],
         "affects": "Young people",
         "controls": [
            "Ensure players wear appropriate footwear with good grip.",
            "Keep the floor dry and clear of any obstacles."
         ]
      },
      {
         "haz

In [19]:
display(Markdown(str(risks["parsed"])))

## Indoor Games at The Queens Hall

### Hazard
Players running into exposed pillars.
### Affects
Young people
### Risks
Injury from collision with pillars, such as bruises or fractures.

Potential for head injuries if players are running fast.
### Controls
Mark the pillars with bright tape to increase visibility.

Set clear boundaries for the play area to avoid the pillars.




### Hazard
Slipping on the hard wooden floor.
### Affects
Young people
### Risks
Injury from falls, such as sprains or fractures.

Potential for head injuries if falling awkwardly.
### Controls
Ensure players wear appropriate footwear with good grip.

Keep the floor dry and clear of any obstacles.




### Hazard
Overcrowding in the play area.
### Affects
All present
### Risks
Increased likelihood of collisions and injuries.

Difficulty in supervising all players effectively.
### Controls
Limit the number of players in each game.

Assign leaders to supervise different areas.




### Hazard
Players getting trapped under stacked chairs.
### Affects
Young people
### Risks
Injury from falling chairs, such as cuts or bruises.

Potential for serious injury if a large stack falls.
### Controls
Ensure chairs are stacked safely and out of play areas.

Supervise the area where chairs are stored.




### Hazard
Inadequate supervision leading to poor behaviour.
### Affects
All present
### Risks
Increased risk of accidents due to rough play.

Potential for bullying or exclusion among players.
### Controls
Ensure enough leaders are present to supervise all activities.

Establish clear rules and consequences for poor behaviour.




### Hazard
Players not following game rules.
### Affects
All present
### Risks
Increased risk of injury due to reckless play.

Potential for arguments or conflicts among players.
### Controls
Explain rules clearly before starting games.

Encourage leaders to intervene if rules are not followed.




### Hazard
Injuries from equipment used in games (e.g., balls).
### Affects
Young people
### Risks
Injury from being hit by a ball, such as bruises or eye injuries.

Potential for accidents if equipment is not used properly.
### Controls
Use soft balls suitable for indoor play.

Inspect equipment before use to ensure it is safe.


